# 📄 Preparação de Dataset para Fine-Tuning
Este notebook processa um arquivo `trn.json` contendo informações de produtos (título e descrição) e gera um novo arquivo `dataset_preparado.jsonl`, no formato ideal para fine-tuning de modelos de linguagem.

In [1]:
import json

### 📌 Caminhos de entrada e saída
Ajuste o caminho abaixo para apontar corretamente para o seu arquivo `trn.json`.

In [2]:
# Caminho completo para o arquivo 'trn.json' descompactado
CAMINHO_JSON = r"D:\Drive\Meu Drive\Cursos - Pós\Fiap - Póstech - 4IADT - IA para Devs\Fase 3 - OpenAI - de 18-03-2025 a 27-05-2025\Tech Challenge\Github\Fase3-Grupo7\LF-Amazon-1.3M\trn.json"

# Caminho onde será salvo o novo arquivo JSONL
SAIDA_JSON = "dataset_preparado.jsonl"


### 🧠 Função para preparar o dataset
Esta função lê linha por linha do arquivo `trn.json`, filtra os dados válidos e grava o resultado no formato `prompt` / `completion`.

In [3]:
def preparar_dataset_jsonl(caminho_entrada, caminho_saida):
    contador = 0
    ignorados = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
         open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            try:
                item = json.loads(linha)
                titulo = item.get("title", "").strip()
                descricao = item.get("content", "").strip()

                if titulo and descricao:
                    prompt = f"O que é {titulo}?"
                    completion = descricao
                    f_out.write(json.dumps({
                        "prompt": prompt,
                        "completion": completion
                    }, ensure_ascii=False) + '\n')
                    contador += 1
                else:
                    ignorados += 1

            except json.JSONDecodeError:
                ignorados += 1
                continue

    print(f"✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {contador}")
    print(f"🚫 Linhas ignoradas (sem conteúdo ou erro): {ignorados}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")


### ▶️ Executar a função

In [4]:
preparar_dataset_jsonl(CAMINHO_JSON, SAIDA_JSON)

✅ Dataset gerado com sucesso!
📄 Linhas válidas: 1390403
🚫 Linhas ignoradas (sem conteúdo ou erro): 858216
💾 Arquivo salvo em: dataset_preparado.jsonl
